In [1]:
!pip install openpyxl

In [2]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

nlp = spacy.load("ja_ginza")
matcher = Matcher(nlp.vocab)
df = pd.read_excel("text/sisyou_db_h29_01.xlsx", skiprows=[1])
df.head()

,ID,年号,年,月,発生\n時間,災害状況,業種（大分類）,Unnamed: 7,業種（中分類）,Unnamed: 9,...,事業場\n規模,起因物（大分類）,Unnamed: 14,起因物（中分類）,Unnamed: 16,起因物（小分類）,Unnamed: 18,事故の型,Unnamed: 20,年齢
0,1,平成,29,1,22～23,工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...,1,製造業,108,化学工業,...,―,3,その他の装置等,37,用具,379,その他の用具,3,激突,54
1,2,平成,29,1,19～20,倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...,8,商業,804,その他の商業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,39
2,3,平成,29,1,18～19,会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...,4,運輸交通業,403,道路貨物運送業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,54
3,4,平成,29,1,16～17,厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。,7,畜産・水産業,701,畜産業,...,10～29,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,414,開口部,1,墜落、転落,50
4,5,平成,29,1,16～17,勤務先の食堂施設内で、ダンボールを束ねてビニールの荷造り紐で縛り結んだ時、手が滑り勢いよく壁...,8,商業,802,小売業,...,50～99,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,419,その他の仮設物、建築物、構築物等,6,激突され,65


In [3]:
patterns = [[{"POS": "NOUN"}] * n for n in [2,3,4]]

## GiNZA 5.0.X
for pattern in patterns:
    name = f'noun_phrase_{len(pattern)}'
    matcher.add(name, [pattern])

## GiNZA 4.0.X
# for pattern in patterns:
#     name = f'noun_phrase_{len(pattern)}'
#     matcher.add(name, None, pattern)

In [4]:
texts = [
    "工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。",
    "倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。",
    "道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。"
]

for doc in nlp.pipe(texts):
    print(doc.text)
    for match_id, begin, end in matcher(doc):
        print(nlp.vocab.strings[match_id], doc[begin:end])
    print('='*80)

工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。
noun_phrase_2 工場内
noun_phrase_2 鉄骨階段
noun_phrase_2 仮組
noun_phrase_2 仮組立
noun_phrase_3 仮組立作業中
noun_phrase_2 組立作業中
noun_phrase_2 右脇腹
noun_phrase_3 右脇腹肋骨
noun_phrase_2 脇腹肋骨
倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。
noun_phrase_2 出入口
noun_phrase_2 冷凍商品
道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。
noun_phrase_2 左車線
noun_phrase_2 大型ワンボックスカー
noun_phrase_2 回転性


In [5]:
from collections import Counter

counter = Counter()
for doc in nlp.pipe(df["災害状況"]):
    nps = [doc[begin:end].text for _, begin, end in matcher(doc)]
    counter.update(nps)

In [6]:
print('count word')
for word, count in counter.most_common(200):
     print('{}\t{}'.format(word,count))

count word
工場内	164
終了後	67
工事現場	56
段目	53
倉庫内	49
作業台	43
配達中	37
歩行中	37
配達先	32
当社工場	30
右手親指	29
大腿部	29
業務中	28
利用者宅	27
清掃中	24
右手中指	23
左手親指	22
右手人差し指	22
左手中指	21
tダンプ	21
お湯	21
当社工場内	20
m下	20
駐車場内	20
資材置場	20
配膳車	20
ため足	19
作業時	19
カゴ車	19
作業終了	19
左手薬指	19
厨房内	19
左手小指	18
tトラック	18
新築工事	18
作業終了後	18
被災労働者	18
左手人差し指	17
利用者様	17
バックヤード	17
お客様宅	17
清掃作業中	17
トラック荷台	16
朝刊配達	16
新聞配達	16
カゴ台車	16
荷台上	15
朝刊配達中	14
左足親指	14
右手小指	14
清掃作業	14
パワーゲート	14
カッターナイフ	14
H鋼	14
従事中	13
切断中	13
完了後	13
事業所内	13
介助中	13
作業現場	13
本社工場	13
場内	12
作業後	12
開口部	12
段ボール	12
包装機	12
ベルトコンベア	12
ミキサー車	11
玄関前	11
業務終了	11
業務終了後	11
反対側	11
敷地内駐車場	11
入所者	11
店舗内	11
製造工場	10
現場内	10
新築工事現場	10
ハンドル操作	10
入浴介助	10
右手薬指	10
左第	10
室内	10
階段下	10
解体工事	10
右大腿	10
ダンプ荷台	9
改修工事	9
対向車線	9
路面凍結	9
左大腿	9
切断作業	9
新聞配達中	9
肋骨骨折	9
包装室	9
cm程度	9
圧迫骨折	9
加工中	9
天井クレーン	9
店舗厨房	9
プラスチック製	9
交換作業	9
安全カバー	9
右大腿部	9
送迎車	9
リン木	9
会社敷地内	8
右脇腹	8
洗車機	8
店舗駐車場	8
事務所内	8
右側面	8
左大腿部	8
バックルーム	8
従業員用	8
右手第	8
誘導警備	8
親指付け根	8
道路上	8
誘導中	8
資材置き場	8
ご利用者	8
左手指	8
左足甲	8
予想以上	8
最上部	8
足場板	8
左側面	8
右足甲	8
右肋骨	8
移動式	8
停車中	